In [18]:
import torch
import torch.nn.functional as F

# t = torch.Tensor([[1.9392, -1.9266, 0.9664], 
#             [0.0000, -1.9266, 0.9664], 
#             [0.0000, -0.0000, 0.9664]]) 

def covariance_formula(ret):
    idx = torch.tensor([[0, 1, 2, 1, 2, 2], [0, 1, 2, 0, 0, 1]], dtype=torch.int64)
    A = torch.zeros((3, 3))
    # ret = torch.rand((6,))# if False else torch.tensor([0, 0, 0, -torch.inf, -torch.inf, -torch.inf], dtype=torch.float32)
    # print(ret.dtype)
    A[idx[0], idx[1]] = ret.exp()
    S = A @ A.T

    loss = 0.5 * ((2 * ret).exp().sum() - 2 * ret[:3].sum() - 3)
    test_loss = 0.5 * (torch.trace(S) - torch.log(torch.linalg.det(S)) - 3)
    assert torch.isclose(loss, test_loss)
    return loss

def naive_batch_covariance_formula(ret):
    losses = []
    for i in range(len(ret)):
        loss = covariance_formula(ret[i])
        losses.append(loss)
    return torch.stack(losses, dim=0)

def test_covariance_formula(input):
    idx = torch.tensor([[0, 1, 2], [0, 1, 2]], dtype=torch.int64)

    # TODO: add requires grad.
    A = torch.zeros((input.shape[0], 3, 3))
    # ret = torch.rand((6,))# if False else torch.tensor([0, 0, 0, -torch.inf, -torch.inf, -torch.inf], dtype=torch.float32)
    # print(ret.dtype)
    A[:, idx[0], idx[1]] = input.exp().sqrt()
    S = A @ A.transpose(1, 2).contiguous()

    loss = 0.5 * (input.exp().sum(dim=1) - input[:, :3].sum(dim=1) - 3)
    test_loss = 0.5 * (S.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1) - torch.log(torch.linalg.det(S)) - 3)
    real_loss = 0.5 * torch.sum(input.exp() - 1 - input, dim = 1)
    print(f'loss={loss}\ntest_loss={test_loss}\nreal_loss={real_loss}')
    # assert torch.allclose(loss, test_loss) and torch.allclose(loss, real_loss)
    return loss

def batch_covariance_formula(input):
    idx = torch.tensor([[0, 1, 2, 1, 2, 2], [0, 1, 2, 0, 0, 1]], dtype=torch.int64)

    # TODO: add requires grad.
    A = torch.zeros((input.shape[0], 3, 3))
    # ret = torch.rand((6,))# if False else torch.tensor([0, 0, 0, -torch.inf, -torch.inf, -torch.inf], dtype=torch.float32)
    # print(ret.dtype)
    # print(f'input={input}')
    A[:, idx[0], idx[1]] = (0.5 * input).exp()
    S = A @ A.transpose(1, 2).contiguous()

    loss = 0.5 * (input.exp().sum(dim=1) - input[:, :3].sum(dim=1) - 3)
    test_loss = 0.5 * (S.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1) - torch.log(torch.linalg.det(S)) - 3)
    # print(f'loss={loss}, test_loss={test_loss}')
    assert torch.allclose(loss, test_loss)
    return loss

batch_size = 8
for i in range(100):
    input = torch.rand((batch_size, 6))
    # naive_output = naive_batch_covariance_formula(input)
    # output = batch_covariance_formula(input)
    loss = batch_covariance_formula(input)
    print(torch.max(loss))
    # assert torch.max(loss) < 1.0
    # print(loss)
    # assert loss < 1.0
# assert torch.allclose(output, naive_output)
# naive_output, output

tensor(3.4232)
tensor(4.2545)
tensor(3.7016)
tensor(3.4406)
tensor(3.6466)
tensor(3.4009)
tensor(3.4254)
tensor(3.8549)
tensor(3.3197)
tensor(3.6359)
tensor(3.6964)
tensor(3.5554)
tensor(4.3252)
tensor(3.7446)
tensor(3.6028)
tensor(3.3352)
tensor(3.6083)
tensor(3.4957)
tensor(3.3896)
tensor(3.8903)
tensor(3.5744)
tensor(3.5507)
tensor(4.2225)
tensor(3.6068)
tensor(4.3965)
tensor(3.3827)
tensor(3.5344)
tensor(3.3113)
tensor(4.0614)
tensor(3.1991)
tensor(3.9012)
tensor(3.6158)
tensor(3.7839)
tensor(4.2452)
tensor(3.4515)
tensor(3.6402)
tensor(3.6038)
tensor(4.1108)
tensor(3.2431)
tensor(3.7444)
tensor(3.6285)
tensor(3.3500)
tensor(4.1990)
tensor(3.4036)
tensor(3.3455)
tensor(3.1004)
tensor(3.4095)
tensor(3.4243)
tensor(3.7540)
tensor(3.5269)
tensor(3.4295)
tensor(4.2171)
tensor(3.4385)
tensor(3.9753)
tensor(3.2329)
tensor(4.1765)
tensor(3.6191)
tensor(3.2938)
tensor(3.3205)
tensor(3.4830)
tensor(3.5335)
tensor(3.7939)
tensor(3.3694)
tensor(3.2330)
tensor(3.6212)
tensor(3.0571)
tensor(4.1